# GOOGLE MAP API

# IMPORT LIBS

In [1]:
import pandas as pd
import json
import requests
import time

# OPEN DATASET

In [2]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv')
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance (miles)
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.639764
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,0.863167
2,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,1.737223
3,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.241710
4,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1,2.353742
...,...,...,...,...,...,...,...,...,...
1789521,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1,0.256146
1789522,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1,0.615670
1789523,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1,2.050854
1789524,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3,3.177537


# GET MY API KEY

In [5]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
API_key = get_keys("../../ASSETS/GMAP/api_key.json")
google_key = list(API_key.values())[0]

# GET THE DRIVING AN DURATION DISTANCE AND INSERT INTO LIST

In [6]:
lat_origin = df['pickup_latitude'].tolist()
long_origin = df['pickup_longitude'].tolist()
lat_destination = df['dropoff_latitude'].tolist()
long_destination = df['dropoff_longitude'].tolist()

distance = []
duration = []
drop = []
for i in range(len(long_destination)):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={lat_origin[i]},{long_origin[i]}&destinations={lat_destination[i]}%2C{long_destination[i]}&key={google_key}"
    print(url)
    r=requests.get(url)
    print(r)
    data = r.json()
    print(data)
    try:
        print(data['rows'][0]['elements'][0]['distance']['text'])
        distance.append(data['rows'][0]['elements'][0]['distance']['text'])
        print(data['rows'][0]['elements'][0]['duration']['text'])
        duration.append(data['rows'][0]['elements'][0]['duration']['text'])
    except:
        drop.append([lat_origin[i], long_origin[i], lat_destination[i], long_destination[i]])
        pass
    time.sleep(0.5)

# CLEAN THE RESULT AND INSERT INTO DF

In [7]:
distance_cleaned = []
duration_cleaned = []
for i in range(len(distance)):
    distance_clean = distance[i].replace(' mi', '').replace(' ft', '')
    float_distance_clean = float(distance_clean)
    distance_cleaned.append(float_distance_clean)
    duration_clean = duration[i].replace(' mins', '').replace(' min', '')
    float_duration_clean = float(duration_clean)
    duration_cleaned.append(float_duration_clean)


df['Driving distance (miles)'] = distance_cleaned
df['Driving duration'] = duration_cleaned
df.head()

# EXPORT DF

In [8]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01a_geo_gmap.csv', index=False)